In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

import requests
import json

import os
import pandas as pd
import numpy as np

from bs4 import BeautifulSoup

import boto3

from IPython.display import HTML, display

In [ ]:
#quick warmup

# imagine a key-value data container
inventory = {
      "001": [{"name": "Milk", "quantity": 34, "price": 1.99}],
      "002": [{"name": "Bread", "quantity": 20, "price": 2.49},
              {"name": "Nutella", "quantity": 5, "price": 2.49}]    
    }



In [ ]:
#How would a store manager find out how much is milk

#How do we add new item? What do we need to do?

In [ ]:
name_to_item = {}
for item_id, details_list in inventory.items():
    for details in details_list:
        # Here we are assuming product names are unique
        name_to_item[details["name"]] = {"id": item_id, "quantity": details["quantity"], "price": details["price"]}
#if details["name"] is not unique, what happens?
print(name_to_item)

# Lecture 05 - JSON, XML, HTML, Requests and APIs
by Jan Šíla, Vítek Macháček <br />
March 26, 2024

### Contents

* Standardized data representation
* JSON
* XML
* Introduction to BeautifulSoup
* Basics of HTML (+ Element Inspection)
* Introduction to Requests (GET vs. POST) and APIs


### Goals:
    
* work with data  online/real-time data
* acquisition, processing - > results

## Date exchange formats - JSON, XML

`Language of the internet`

* You can send/receive a message with (almost) any service

* send .docx -> what if I do not have MS Word?
* we need a simple data format which would work on any machine (system agnostic), is general (can write anything) and is ediatable in basic editors

* More complex than simple tables
* Highly structured - if you dont follow the rules, you are out
* Both sides need to understand the structure (comments in yaml)
* only data, no code to be run (security measure)
* distributed as text/string (to be precise as `bytes` literals) 
* parsed to objects - easy to work with straight away
* Can be persisted as special files, or some data streams from APIs. 
* Human readable
* Hierarchical
* Can be fetched using standard web APIs

### Purpose

1. Communication 
    * All imaginable communication channels
    * Applications within single server/machine
    * Only transferring of data
    * Both sides need to understand the structure

2. Storing
    * self-descriptive
    * human readable
    * also in DBs - SQL, MongoDB etc.

3. Standardization
    * predictability
    * cooperation
    * spillovers from standardization


### Dimensionality problem

* rich information comes at costs of data complexity 
* to interrelate information, you need to high dimensionality (or A LOT of columns) or declaratory formats such as protobuf
* Strongly object-oriented


### 1D:
* logs

### 2D: CSVs
* tabular data (like pandas DFs)

### 3+D:
#### XML
* eXtensible Markup Language is a software- and hardware-independent tool for storing and transporting data.
* Officialy defined at 1998, but its roots are even older.
* XML was designed to carry data - with focus on what data is
* HTML was designed to display data - with focus on what data should look like displayed 
* XML tags are not predefined like HTML tags are
* more verbose than JSON
* can have comments !actually a really cool in useful feature!
* used historically as a transaction format in many areas: 
    * Scientific measurements
    * News information
    * Wheather measurements
    * Financial transactions
* Necessary to use XML parser to use in Python or in JavaScript


### JSON
* JavaScript Object Notation
* often *.json* files
* but also used in the web etc.
* supports standard datatypes - strings, integers, floats, lists
* No comments
* More compact, less verbose
* No closing tags
* Used EVERYWHERE, BUT [NOT LICENSED FOR EVIL](https://www.json.org/license.html). If you want to do evil stuff, use XML instead.
* Native in JavaScript and close to native in Python (dictionary)
* Jupyter Notebooks


* common pitfals: properly formatted JSON is different to python dict. -> check: https://jsonlint.com/


# JSON

In [ ]:
# general representation of a dictionary
# emphasis on accessibility -> key-value ( hash table )
# contains records, lists, or other dictionaries

teachers = [
    {'name':'Jozef Baruník','titles':['doc.','PhDr.','Ph.D.','Bc.','Mgr.'],'ID':1234,'courses':['JEM005','JEM116','JEM059','JEM061']},
    {'name':'Martin Hronec','titles':['Bc.','Mgr.'],'ID':3421,'courses':['JEM005','JEM207']},
]

courses = {
    "JEM005":{'name':'Advanced Econometrics','ECTS':6,'teachers':[3421,1234]},
    'JEM207':{'name':'Data Processing in Python','ECTS':5,'teachers':[3421]},
    'JEM116':{'name':'Applied Econometrics','ECTS':6,'teachers':[1234]},
    'JEM059':{'name':'Quantitative Finance I.','ECTS':6,'teachers':[1234,5678]},
    'JEM061':{'name':'Quantitative Finance II.','ECTS':6,'teachers':[1234,5678]}
}
jsondata = {'teachers':teachers,'courses':courses}
jsondata

is this a valid JSON?

https://jsonformatter.curiousconcept.com/

![python and JSON](./06_pics/python_json.png)

In [ ]:
js = json.dumps(
    jsondata
) #json formatted string!

isinstance(js,str)


In [ ]:
js

In [ ]:
jsondata['courses']['JEM005']['test']='test'

In [ ]:
pd.DataFrame(jsondata['courses']).transpose()

In [ ]:
dfc = pd.read_json(json.dumps(jsondata['courses']),orient='index')
dfc

In [ ]:
# lets come back to this a little later

# eXtensible Markup Language (XML)

* elements
* attributes
* tags

### Tag
> <>

### Element

### Convert to python data-types

In [ ]:
#either
'''<element>content</element>'''

#or self-closing (no content)
'''<element />''';
# <br />

### Attributes

In [ ]:
'''<element attr="value" />''';

![XML tree structure](./06_pics/xml_tree_structure.png)

```xml
<bookstore>
    <book category="fiction">
        <title lang="ENG">Everyday Italian</title>
        <title lang="CZE">AAaAA</title>
        <author>Giada De Laurentis</author>
        <year>2005</year>
        <price>30.00</price>
    </book>
</bookstore>
```

```json
{
    "bookstore":[
        {
            "title":"Everyday Italian",
            "lang":"ENG",
            "author":"Giada de Laurentis",
            "year":2005,
            "price":30
        }
    ]
}
```


Takeaway: JSON and XML are not equivalents and cannot be freely mirrored. Unfortunately.

JSON cannot have multiple tags with different properties ->title_en, title_cze  perhaps

## Navigation
* Xpath
* CSS selectors 
* **BeautifulSoup**

### BeatifulSoup in detail
each BS object represents
* an element
* the position in tree

In [ ]:
'''string  on more 
nes '''

In [ ]:
xml = '''
<?xml version="1.0" encoding="utf-8"?>
<ies_data>
    <courses>
        <course id="JEM005" ects="6" name="Advanced Econometrics">
           <teacher-id>3421</teacher-id>
           <teacher-id>1234</teacher-id>
        </course>
        <course id="JEM207" ects="5" name="Data Processing in Python">
            <teacher-id>3421</teacher-id>
        </course>
            <course id="JEM116" ects="6" name="Applied Econometrics I.">
            <teacher-id>1234</teacher-id>
        </course>
        <course id="JEM059" ects="6" name="Quantitative Finance I.">
            <teacher-id>1234</teacher-id>
            <teacher-id>5678</teacher-id>
        </course>
        <course id="JEM061" ects="6" name="Quantitative Finance II.">
            <teacher-id>1234</teacher-id>
            <teacher-id>5678</teacher-id>
        </course>
    </courses>
    <teachers>
        <teacher teacher-id="3421">
            <name>Martin Hronec</name>
        </teacher>
        <teacher teacher-id="1234">
            <name>Jozef Baruník</name>
        </teacher>
        <teacher teacher-id="5678">
            <name>Lukáš Vácha</name>
        </teacher>
    </teachers>
</ies_data>
'''

#unlike HTML, those tag names are defined by Vitek - no one else 'can' understand them -> flexibility is limited. But same issue with JSON to be fair

soup = BeautifulSoup(xml)

In [ ]:
dir(soup)

```find()``` will find a **first** element given the input

```find_all()``` or ```findAll()```  finds a **all** elements given the input

In [ ]:
soup.find_all('course')[0].find('teacher-id')

In [ ]:
jem059 = soup.find('course',{'id':'JEM059'}) #looking for a tag with attrbitues (optional)


In [ ]:
jem059

In [ ]:
soup.findAll('teacher-id')

`soup['attr']` will return the value of attribute

In [ ]:
print(jem059['ects'])
print(jem059['name'])

In [ ]:
soup.findAll('teacher-id')

In [ ]:
jem059

you can also navigate horizontally

In [ ]:
jem059.findNext('course').findNext('course')

In [ ]:
jem059.findPrevious('course').findPrevious('course')

and even upstream!

In [ ]:
jem059.parent.parent

In [ ]:
#get all teacher ids
teacher_ids = [int(t.text) for t in soup.findAll('teacher-id')]
print(teacher_ids)
#get unique
set(teacher_ids)

In [ ]:
course = soup.find('course')
d = {
    'id':course['id'],
    'name':course['name'],
    'ects':course['ects'],
    'teachers':[int(t.text) for t in course.findAll('teacher-id')]
}
d

### Can convert to JSON-like

In [ ]:
l = []
for course in soup.findAll('course'):
    d = {'id':course['id'],
         'name':course['name'],
         'ects':course['ects'],
         'teachers':[int(t.text) for t in course.findAll('teacher-id')]}
    l.append(d)
l

### Or in list-comprehension syntax

In [ ]:
l = [{
    'id':course['id'],
    'name':course['name'],
    'ects':course['ects'],
    'teachers':[int(t.text) for t in course.findAll('teacher-id')]
} for course in soup.findAll('course')]

In [ ]:
l

In [ ]:
pd.DataFrame(l)

# HTML
standard web-page consists of:

* Browser-executed code (`front-end`)
    * HTML "DOM" structure - the website content
        * List of elements that are on website
        * Links to CSS classes, ids and
    * CSS stylesheets - website graphics
    * JavaScripts - website interactivity    

* Server-executed (`back-end`)
    * Server, database, app logic etc.
    * Not available for scraping!
    * May be available as API


## Web-scraping
* client side only
* Navigating HTML DOM by taking advantage of CSS structure

## DOM (Document Object Module):

In [ ]:
html = '''
<html>
    <head>
        <title>Sample page</title>
    <script>
        function click_button() {
            alert('Hi guys, still enjoying Python?!')
        }
    </script>
    <style>
        #content div {
            color:black;
        }
        .firstRow {
            background-color:#ddd;
        }

        .normalRow {
            background-color:white;
        }
    </style>
    </head>
    
    <body>
        <div id="header">
            My page header
        </div>
        <div id="table_container">
            <table>
                <tr class="firstRow">
                    <td>name</td>
                    <td>number</td>
                </tr>
                <tr class="normalRow">
                    <td>B</td>
                    <td>2</td>
                </tr>
                <tr class="normalRow">
                    <td>C</td>
                    <td>3</td>
                </tr>
            </table>
        </div>
        <div id="button_container">
            <button id="btn" onclick="click_button()">Click Me!</button>
        </div
    </body>
</html>
'''
display(HTML(html))

In [ ]:
soup = BeautifulSoup(html,'html')
soup

In [ ]:
rows = soup.findAll('tr',{'class','normalRow'})
rows

In [ ]:
d = {}

for row in rows:
    key = row.findAll('td')[0].text
    val = int(row.findAll('td')[1].text)
    d[key] = val
pd.Series(d)

In [ ]:
d

In [ ]:
pd.Series({
    row.findAll('td')[0].text:int(row.findAll('td')[1].text) 
    for row in BeautifulSoup(html).findAll('tr',{'class':'normalRow'})})

In [ ]:
soup = BeautifulSoup(html)

In [ ]:
row = soup.findAll('tr',{'class':'normalRow'})[0]

In [ ]:
row

In [ ]:
row.findAll('td')[0].text

In [ ]:
int(row.findAll('td')[1].text)

In [ ]:
{row.findAll('td')[0].text:int(row.findAll('td')[1].text) for row in soup.findAll('tr',{'class':'normalRow'})}

## HTML Inspection
http://ies.fsv.cuni.cz/cs/node/51

In [ ]:
import requests

# requests and internet communication

* `Client` asks/requests questions (your Jupyter client)
* `Server` replies/serve answers (your Jupyter server)


API = *Application Programming Interface*

very general term! Not only used in web communication

## HTTP requests

A most standard webserver communication channel around

A standard HTTP request contains:

* URL 

    * domain
    * route
    * parameters

* Request Type - GET, POST, PUT, DELETE (see below)

* Content specification - 
    * Application/JSON
    * Application/XML
    * text/html
    * text/css

* Content

* Outcoming data (will see below)

* Cookies 

* Status Code:

    * 200 - success
    * 404 - resource does not exist
    * 500 - the server failed during processing your request


1) REST API - use HTTP request and returns JSON

2) SOAP API - use HTTP request and returns XML

3) Website - use HTTP request and returns set of HTML, JavaScript, CSS and other files

### When to use?
* whenever more applications need to communicate
* user-friendly interface for complicated tasks - DEEP AI, Google Maps
* Data - Golemio, OpenStreetMaps

### GET request
* fast
* public
* data flow only one direction
* parameters via request adress

> https://www.google.com/search?q=how+to+understand+url+parameters&rlz=1C1GCEU_csCZ860CZ860&oq=how+to+understand+url+parameters&aqs=chrome..69i57j33i22i29i30l7.5237j0j4&sourceid=chrome&ie=UTF-8


In [ ]:
r = requests.get('https://cs.wikipedia.org/wiki/Institut_ekonomick%C3%BDch_studi%C3%AD_Fakulty_soci%C3%A1ln%C3%ADch_v%C4%9Bd_Univerzity_Karlovy')
#plain request - like browser
r.text

In [ ]:
soup = BeautifulSoup(r.text,'html')
tags=soup.findAll('span', {'class':"wd"})

In [ ]:
tags

### POST request
* slow
* private
* both sides can send data

## Static pages x Dynamic pages x JavaScript-rendered pages

### Static

* pages that do not get updated instantly
* all information necessary for rendering a website is available after entering the URL
* It may ask the database, but the output is stable.
* all parameters within the adress!
* Typical example:
    
### JavaScript rendered: 
* Defacto static, but you cannot take advantage of HTML/CSS structure

### Dynamic content
* webpage instantly communicates with the webserver and the database
* 
* solution -> Selenium!

### Is this website static or dynamic?

1. Facebook
2. Sreality.cz
3. IES website



## How to chose data source for project

You need to know in advance what data you will download:

1. full or satisfactory access to API
2. the web-page is parsable (prefer not too much javascript)
3. plan to generate all requests

# APIs Example
### Get wiki data using GET

In [ ]:
#if time, return to geodata

# Lets start with a basic request

In [ ]:
api_url = 'https://krcgc3uqga.execute-api.eu-central-1.amazonaws.com'
#this api implements three routers
# GET /time
# GET /stocks
# POST /hashme

In [ ]:
route = 'time'
# route /ruːt/
response = requests.get(f'{api_url}/{route}')
response.json()


In [ ]:
# route = stocks

route = 'stocks'
# route /ruːt/
response = requests.get(f'{api_url}/{route}')
print(response.json())

In [ ]:
route = "hashme"
url = f"https://krcgc3uqga.execute-api.eu-central-1.amazonaws.com/{route}"

payload = json.dumps({
  "name": "Jan Sila"
})
headers = {
  'Content-Type': 'application/json'
}

response = requests.post(url, headers=headers, data=payload)

print(response.json())


In [ ]:
response = requests.get('https://en.wikipedia.org/wiki/Charles_University')
soup = BeautifulSoup(response.text)
div = soup.find('div',{'id':'mw-content-text'}) #  #mw-content-text > div > p:nth-child(10)texts)
article = ' '.join([p.text for p in div.find_all('p')])
print(article)

# Bonus example:

## GeoJSON

* One standardized data format for transferring geodata
* Plenty of geodata out there
* see for example http://opendata.iprpraha.cz/CUR/OVZ/OVZ_Klima_ZnecOvzdusi_p/WGS_84/OVZ_Klima_ZnecOvzdusi_p.json


In [ ]:

verbose_request = requests.get('http://opendata.iprpraha.cz/CUR/OVZ/OVZ_Klima_ZnecOvzdusi_p/WGS_84/OVZ_Klima_ZnecOvzdusi_p.json')


In [ ]:
print(verbose_request.status_code)
dir(verbose_request)


In [ ]:
verbose_request.json()



In [ ]:
#get already json
d = requests.get('http://opendata.iprpraha.cz/CUR/OVZ/OVZ_Klima_ZnecOvzdusi_p/WGS_84/OVZ_Klima_ZnecOvzdusi_p.json').json()

d['features'][0]['properties']

In [ ]:
import branca
import folium

colorscale = branca.colormap.linear.YlOrRd_09.scale(0, 5)

def style_function(feature):
    gridvalue = feature['properties']['GRIDVALUE']
    return {
        'fillOpacity': 0.5,
        'weight': 0,
        'fillColor': colorscale(gridvalue)
    }

m = folium.Map(location=[50.085,14.45],zoom_start=11)
folium.GeoJson('http://opendata.iprpraha.cz/CUR/OVZ/OVZ_Klima_ZnecOvzdusi_p/WGS_84/OVZ_Klima_ZnecOvzdusi_p.json',style_function=style_function).add_to(m)
m